# Train model

In [12]:
import glob
import os
# Set cache to external hd
os.environ['TRANSFORMERS_CACHE'] = '/home/leonardovida/data/volume_1/huggingface_cache/'
os.environ['HF_DATASETS_CACHE'] = '/home/leonardovida/data/volume_1/huggingface_cache/'
from pathlib import Path

from typing import Any, Dict, List, NamedTuple, Optional, Sequence, Tuple, Union

from tokenizers import Tokenizer
from tokenizers import ByteLevelBPETokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing
from transformers import RobertaConfig
from transformers import RobertaTokenizerFast
from transformers import BertTokenizer
from transformers import PreTrainedTokenizer
from transformers import TrainingArguments
from transformers import RobertaForMaskedLM
from datasets import Dataset, DatasetDict
from datasets import load_from_disk

import torch
from torch import Tensor
from torch.utils.data import DataLoader

import numpy as np
import pandas as pd
from tqdm.notebook import trange, tqdm

In [1]:
!nvidia-smi

Fri Apr 30 12:20:54 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 455.32.00    CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  On   | 00000000:00:05.0 Off |                  N/A |
| 30%   28C    P8     9W / 250W |      1MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Set environment variables

In [2]:
#PATH_RAW_FILES = "/home/leonardovida/data-histaware/raw/raw_merged/"
PATH_RAW_FILES = "/home/leonardovida/data/volume_1/data-histaware/merged_articles/1970s"
#PATH_TOKENIZER_DIR = "/home/leonardovida/dev/hist-aware/notebooks/models/bert-training-from-scratch/tokenizer"
PATH_TOKENIZER_DIR = "/home/leonardovida/data/volume_1/data-histaware/tokenizer"
#PATH_DATASET_DIR = "/home/leonardovida/dev/hist-aware/notebooks/models/bert-training-from-scratch/dataset"
PATH_DATASET_DIR = "/home/leonardovida/data/volume_1/data-histaware/dataset"

# For tokenizer
PRE_TRAINED_MODEL_NAME = 'wietsedv/bert-base-dutch-cased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)
#!mkdir PATH_MODEL_DIR

dataset = load_from_disk(PATH_DATASET_DIR)

os.environ["CUDA_LAUNCH_BLOCKING"]='1'  #Makes for easier debugging (just in case)

weights_dir = "/home/leonardovida/data/volume_1/data-histaware/weights"
#!mkdir {weights_dir}

In [19]:
torch.cuda.is_available()

True

In [5]:
config = RobertaConfig(
    vocab_size=30_000,
    max_position_embeddings=512,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1, # The vocabulary size of the token_type_ids passed
)

Paper BERTje
--max_predictions_per_seq=20 \
  --train_batch_size=256 \
  --eval_batch_size=32 \
  --learning_rate=1e-4 \
  --num_train_steps=1000000 \
  --num_warmup_steps=10000 \
  --save_checkpoints_steps=10000 \
  --iterations_per_loop=10000 \
  --max_eval_steps=10000 \

In [17]:
tokenizer

PreTrainedTokenizer(name_or_path='wietsedv/bert-base-dutch-cased', vocab_size=30000, model_max_len=512, is_fast=False, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [7]:
model = RobertaForMaskedLM(config=config)

In [8]:
model.num_parameters()
# => 84 million parameters

66584880

### Tokenizing the dataset

In [16]:
def tokenize_function(article):
    # Remove empty lines
    article["sentences"] = [line for line in article["sentences"] if len(line) > 0]
    return tokenizer(
        article["sentences"],
        padding=True,
        truncation=True,
        max_length=250,
        # We use this option because DataCollatorForLanguageModeling (see below) is more efficient when it
        # receives the `special_tokens_mask`.
        return_special_tokens_mask=True,
    )

tokenized_datasets = dataset.map(
    tokenize_function,
    batched=True,
    num_proc=9,
    remove_columns=["p", "p_clean", "sentences", "subject", "recordIdentifier", "sents_len"],)

ValueError: too many values to unpack (expected 2)

In [10]:
def encode(article):
    return tokenizer(
        article["sentences"],
        truncation=True,
        padding='max_length',
        # TODO: padding 
        max_length=512,
    )

dataset = dataset.map(encode, batched=True)
dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

dataloader = torch.utils.data.DataLoader(dataset, batch_size=4)
next(iter(dataloader))

ValueError: too many values to unpack (expected 2)

In [37]:
def prepare_data(dataset_f: str,
                 tokenizer: PreTrainedTokenizer,
                 batch_size: int = 16,
                 num_workers: int = 2) -> Dict[str, DataLoader]:
    """Given an input file, prepare the train, test, validation dataloaders.
       The created datasets will be preprocessed and save to disk.
    :param dataset_f: input file
    :param tokenizer: pretrained tokenizer that will prepare the data, i.e. convert tokens into IDs
    :param batch_size: batch size for the dataloaders
    :param num_workers: number of CPU workers to use during dataloading. On Windows this must be zero
    :return: a dictionary containing train, test, validation dataloaders
    """

    def collate(batch: List[Dict[str, Tensor]]) -> Dict[str, Tensor]:
        """Collates gathered items to form a batch which is then used in training, evaluation, or testing.
        :param batch: a list of samples from the dataset. Each sample is a dictionary with keys "input_ids".
        :return: the created batch with keys "input_ids"
        """
        all_input_ids = pad_sequence([item["input_ids"] for item in batch]).to(torch.long)

        return {"input_ids": all_input_ids}

    def preprocess(sentences: List[str]) -> Dict[str, Union[list, Tensor]]:
        """Preprocess the raw input sentences from the text file.
        :param sentences: a list of sentences (strings)
        :return: a dictionary of "input_ids"
        """
        tokens = [s.split() for s in sentences]

        # The sequences are not padded here. we leave that to the dataloader in collate
        # That means: a bit slower processing, but a smaller saved dataset size
        return tokenizer(tokens,
                         add_special_tokens=False,
                         return_token_type_ids=False,
                         return_attention_mask=False
                        )
    
    dataset = Dataset.from_dict({"text": Path(dataset_f).read_text(encoding="utf-8").splitlines()})

    # Split the dataset into train, test, dev
    # 90% (train), 10% (test + validation)
    train_testvalid = dataset.train_test_split(test_size=0.1)
    # 10% of total (test), 10% of total (validation)
    test_valid = train_testvalid["test"].train_test_split(test_size=0.5)

    dataset = DatasetDict({"train": train_testvalid["train"],
                           "test": test_valid["test"],
                           "valid": test_valid["train"]})

    dataset = dataset.map(preprocess, input_columns=["text"], batched=True)
    dataset.set_format("torch", columns=["input_ids"])

    return {partition: DataLoader(ds,
                                  batch_size=batch_size,
                                  shuffle=True,
                                  collate_fn=collate,
                                  num_workers=num_workers,
                                  pin_memory=True) for partition, ds in dataset.items()}

In [38]:
prepare_data(dataset_f = "/home/leonardovida/data-histaware/articles0_50000.txt",
    tokenizer = tokenizer,
    batch_size = 16,
    num_workers = 2)

TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]

### Create Data Collator

To make backpropagation on

In [25]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

### Initalize Trainer

In [26]:
training_args = TrainingArguments(
    output_dir="home/leonardovida/data-histaware/delphBERT",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

NameError: name 'dataset' is not defined

In [ ]:
%%time
trainer.train()

#### Save model

In [ ]:
trainer.save_model("./EsperBERTo")

### Verify learning

Create a fill-mask pipeline to check whether the model learned anything useful

In [ ]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./EsperBERTo",
    tokenizer="./EsperBERTo"
)

In [ ]:
fill_mask("La suno <mask>.")

### Upload the model to HuggingFace

Write a README.md model card and add it to the repository under `model_cards/`. Your model card should ideally include:
* a model description,
* training params (dataset, preprocessing, hyperparameters), 
* evaluation results,
* intended uses & limitations
* whatever else is helpful! 🤓

In [ ]:
#transformers-cli upload